simple gen ai app using langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
#langsmith trackking
os.environ['LANGCHAIN_API_KEY']=os.environ("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2']=True
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

From the website we need to scrape the data

In [4]:
from langchain_community.document_loaders import WebBaseLoader
#scrape the entire content fron the website
loader=WebBaseLoader("https://medium.com/around-the-prompt/what-is-langsmith-and-why-should-i-care-as-a-developer-e5921deb54b5")
loader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://medium.com/around-the-prompt/what-is-langsmith-and-why-should-i-care-as-a-developer-e5921deb54b5', 'title': 'What is LangSmith and why should I care as a developer? | by Logan Kilpatrick | Around the Prompt | Medium', 'description': 'I have said it before and I will say it again, the tooling around large language models (LLM’s) is still in its infancy. Due to the nature of LLM’s and their inherent dynamism, traditional software…', 'language': 'en'}, page_content='What is LangSmith and why should I care as a developer? | by Logan Kilpatrick | Around the Prompt | MediumSitemapOpen in appSign upSign inMedium LogoWriteSign upSign inAround the Prompt·‘Around the Prompt’ goes deep, peeling back the layers of AI innovation to reveal the hidden gems, the untapped potential, based on conversations with leading experts.What is LangSmith and why should I care as a developer?A deep dive into the latest product from the creators of LangChain 🦜Logan KilpatrickF

In [8]:
#in llm model there is contect size so we need to split into chnks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
documents=text_splitter.split_documents(docs)

In [9]:
documents

[Document(metadata={'source': 'https://medium.com/around-the-prompt/what-is-langsmith-and-why-should-i-care-as-a-developer-e5921deb54b5', 'title': 'What is LangSmith and why should I care as a developer? | by Logan Kilpatrick | Around the Prompt | Medium', 'description': 'I have said it before and I will say it again, the tooling around large language models (LLM’s) is still in its infancy. Due to the nature of LLM’s and their inherent dynamism, traditional software…', 'language': 'en'}, page_content='What is LangSmith and why should I care as a developer? | by Logan Kilpatrick | Around the Prompt | MediumSitemapOpen in appSign upSign inMedium LogoWriteSign upSign inAround the Prompt·‘Around the Prompt’ goes deep, peeling back the layers of AI innovation to reveal the hidden gems, the untapped potential, based on conversations with leading experts.What is LangSmith and why should I care as a developer?A deep dive into the latest product from the creators of LangChain 🦜Logan KilpatrickF

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()


In [ ]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [ ]:
vectorstoredb

In [ ]:
query=" first place to start is with Langchain"
result=vectorstoredb.similarity_search(query)
result[0].page_content

In [ ]:
#i wnat to give context related to the query using retrievel chains
#document chain, 
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate(
    """ answer the following question based on only provided context:
    <context>
    {context}
    <context>"""
)

In [ ]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [ ]:
Document_chain=create_stuff_documents_chain(llm,prompt)
Document_chain

In [ ]:
from langchain_core.documents import Document
Document_chain.invoke({
    "input":"what is langsmith"
    "context":[Document(page_content="what is langsmith")]})

In [ ]:
#if i passing any input using retriever it will take relevent information from vector db


In [ ]:
#create retriever
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retriever_chain=create_retrieval_chain(retriever,Document_chain)

In [ ]:
retriever_chain

In [ ]:
response=retriever_chain.invoke({"input:what is lang smith"})
response['answer']

In [ ]:
response

In [ ]:
response['context']